In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1" 
import ast
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
from keras import backend as K
from keras import Model
from keras import optimizers
from keras.legacy import interfaces
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Flatten, Activation, GlobalAveragePooling2D, BatchNormalization
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from keras.optimizers import Adam

from keras.optimizers import SGD
from keras import callbacks
from data import *
from keras_helper import *

from keras.applications import Xception
from keras.applications.xception import preprocess_input

Using TensorFlow backend.


In [2]:
TOT = 49673580
EPOCHS = 50
size = 71
batchsize = 256
lw = 2
channel = 3
STEPS = TOT / EPOCHS / batchsize / 3
NCATS = 340

In [3]:
K.clear_session()
base_model = Xception(input_shape=(size, size, channel), weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = layers.Dense(classes, activation='softmax', name='predictions')(x)
predictions = Dense(NCATS, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False
     
# model = Xception(input_shape=(size, size, 1), weights=None, classes=NCATS)

model.compile(optimizer=Adam(lr=0.002), loss='categorical_crossentropy',
              metrics=[categorical_crossentropy, categorical_accuracy, top_3_accuracy])
# print(model.summary())

In [4]:
valid_df = pd.read_csv('../input/valid.csv')
x_valid = df_to_image_array_xd(valid_df, size, lw=lw, preprocess_input=preprocess_input, channel=channel)
y_valid = keras.utils.to_categorical(valid_df.word, num_classes=NCATS)

In [5]:
print(x_valid.shape, y_valid.shape)
print('Validation array memory {:.2f} GB'.format(x_valid.nbytes / 1024.**3 ))

(34000, 71, 71, 3) (34000, 340)
Validation array memory 1.92 GB


In [6]:
train_datagen = image_generator_xd(size=size, batchsize=batchsize, lw=lw, 
                                   preprocess_input=preprocess_input,
                                  channel=channel)

In [7]:
model_prefix = 'xception_imagenet'
fold = 9
callbks = [
    ReduceLROnPlateau(monitor='val_categorical_accuracy', factor=0.5, patience=5,
                      min_delta=0.005, mode='max', cooldown=3, verbose=1),
    callbacks.ModelCheckpoint("./models/{}{}.model".format(model_prefix, size),
                                monitor='val_categorical_accuracy', 
                                mode = 'max', save_best_only=True, verbose=1),
    callbacks.TensorBoard(log_dir='./log/{}{}'.format(model_prefix, size)),
]

In [8]:
hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=8, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

for layer in model.layers:
   layer.trainable = True

hist = model.fit_generator(
    train_datagen, steps_per_epoch=STEPS, epochs=EPOCHS, verbose=1,
    validation_data=(x_valid, y_valid),
    callbacks = callbks
)

Epoch 1/8
1294/1293 [==============================] - 138s 106ms/step - loss: 4.3645 - categorical_crossentropy: 4.3645 - categorical_accuracy: 0.2177 - top_3_accuracy: 0.3598 - val_loss: 7.5861 - val_categorical_crossentropy: 7.5861 - val_categorical_accuracy: 0.0614 - val_top_3_accuracy: 0.1341

Epoch 00001: val_categorical_accuracy improved from -inf to 0.06135, saving model to ./models/xception_imagenet71.model
Epoch 2/8
1294/1293 [==============================] - 135s 104ms/step - loss: 3.9777 - categorical_crossentropy: 3.9777 - categorical_accuracy: 0.2618 - top_3_accuracy: 0.4194 - val_loss: 8.3273 - val_categorical_crossentropy: 8.3273 - val_categorical_accuracy: 0.0619 - val_top_3_accuracy: 0.1351

Epoch 00002: val_categorical_accuracy improved from 0.06135 to 0.06191, saving model to ./models/xception_imagenet71.model
Epoch 3/8
1294/1293 [==============================] - 135s 104ms/step - loss: 3.8589 - categorical_crossentropy: 3.8589 - categorical_accuracy: 0.2758 - top

/home/kownse/anaconda3/lib/python3.6/site-packages/keras/engine/training.py:479: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Epoch 1/50
1294/1293 [==============================] - 135s 104ms/step - loss: 3.5738 - categorical_crossentropy: 3.5738 - categorical_accuracy: 0.3147 - top_3_accuracy: 0.4830 - val_loss: 9.1226 - val_categorical_crossentropy: 9.1226 - val_categorical_accuracy: 0.0623 - val_top_3_accuracy: 0.1317

Epoch 00001: val_categorical_accuracy improved from 0.06191 to 0.06226, saving model to ./models/xception_imagenet71.model
Epoch 2/50
1294/1293 [==============================] - 135s 104ms/step - loss: 3.5647 - categorical_crossentropy: 3.5647 - categorical_accuracy: 0.3162 - top_3_accuracy: 0.4855 - val_loss: 9.5100 - val_categorical_crossentropy: 9.5100 - val_categorical_accuracy: 0.0546 - val_top_3_accuracy: 0.1255

Epoch 00002: val_categorical_accuracy did not improve from 0.06226
Epoch 3/50
1294/1293 [==============================] - 135s 104ms/step - loss: 3.5465 - categorical_crossentropy: 3.5465 - categorical_accuracy: 0.3188 - top_3_accuracy: 0.4876 - val_loss: 9.2827 - val_categ

1294/1293 [==============================] - 135s 104ms/step - loss: 3.4114 - categorical_crossentropy: 3.4114 - categorical_accuracy: 0.3359 - top_3_accuracy: 0.5070 - val_loss: 9.4177 - val_categorical_crossentropy: 9.4177 - val_categorical_accuracy: 0.0549 - val_top_3_accuracy: 0.1277

Epoch 00022: val_categorical_accuracy did not improve from 0.06226
Epoch 23/50
1294/1293 [==============================] - 135s 104ms/step - loss: 3.4065 - categorical_crossentropy: 3.4065 - categorical_accuracy: 0.3367 - top_3_accuracy: 0.5067 - val_loss: 9.3725 - val_categorical_crossentropy: 9.3725 - val_categorical_accuracy: 0.0558 - val_top_3_accuracy: 0.1286

Epoch 00023: val_categorical_accuracy did not improve from 0.06226
Epoch 24/50
1294/1293 [==============================] - 135s 104ms/step - loss: 3.4025 - categorical_crossentropy: 3.4025 - categorical_accuracy: 0.3381 - top_3_accuracy: 0.5079 - val_loss: 9.3245 - val_categorical_crossentropy: 9.3245 - val_categorical_accuracy: 0.0562 - 


Epoch 00043: val_categorical_accuracy did not improve from 0.06226
Epoch 44/50
1294/1293 [==============================] - 135s 104ms/step - loss: 3.3875 - categorical_crossentropy: 3.3875 - categorical_accuracy: 0.3393 - top_3_accuracy: 0.5086 - val_loss: 9.3597 - val_categorical_crossentropy: 9.3597 - val_categorical_accuracy: 0.0550 - val_top_3_accuracy: 0.1289

Epoch 00044: val_categorical_accuracy did not improve from 0.06226
Epoch 45/50
1294/1293 [==============================] - 135s 105ms/step - loss: 3.3889 - categorical_crossentropy: 3.3889 - categorical_accuracy: 0.3410 - top_3_accuracy: 0.5105 - val_loss: 9.3575 - val_categorical_crossentropy: 9.3575 - val_categorical_accuracy: 0.0551 - val_top_3_accuracy: 0.1287

Epoch 00045: val_categorical_accuracy did not improve from 0.06226
Epoch 46/50
1294/1293 [==============================] - 135s 104ms/step - loss: 3.3837 - categorical_crossentropy: 3.3837 - categorical_accuracy: 0.3395 - top_3_accuracy: 0.5099 - val_loss: 9.3

In [9]:
model.loa d_weights('./models/{}{}.model'.format(model_prefix, size))

SyntaxError: invalid syntax (<ipython-input-9-0893867fcc51>, line 1)

In [ ]:
test = pd.read_csv('../input/test_simplified.csv')
test.head()
x_test = df_to_image_array_xd(test, size, lw=lw, 
                              preprocess_input=preprocess_input,
                             channel=channel)
print(test.shape, x_test.shape)
print('Test array memory {:.2f} GB'.format(x_test.nbytes / 1024.**3 ))

In [ ]:
test_predictions = model.predict(x_test, batch_size=128, verbose=1)

top3 = preds2catids(test_predictions)
top3.head()
top3.shape

In [ ]:
np_classes = np.load('../input/classes.npy')
id2cat = {k: cat.replace(' ', '_') for k, cat in enumerate(np_classes)}
top3cats = top3.replace(id2cat)
top3cats.head()
top3cats.shape

In [ ]:
test['word'] = top3cats['a'] + ' ' + top3cats['b'] + ' ' + top3cats['c']
submission = test[['key_id', 'word']]
submission.head()

In [ ]:
import kaggle_util
kaggle_util.save_result(submission, 
                        '../result/{}{}.csv'.format(model_prefix, size), 
                        'quickdraw-doodle-recognition', 
                        send=True, index=False)